In [117]:
import UnityPy
from PIL import Image
from pathlib import Path
import os
import shutil

In [118]:
# Load
env = UnityPy.load(f'../data/ui_jacket_001686.ab')

In [119]:
env

In [120]:
def print_structure(obj, path="root"):
    if isinstance(obj, dict):
        for key, value in obj.items():
            current_path = f"{path}.{key}"
            print_structure(value, current_path)
    elif isinstance(obj, list):
        for index, value in enumerate(obj):
            current_path = f"{path}[{index}]"
            print_structure(value, current_path)
    else:
        print(f"{path}: {obj}")

def recursive_string_replace(obj, old, new):
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = recursive_string_replace(value, old, new)
    elif isinstance(obj, list):
        for index, value in enumerate(obj):
            obj[index] = recursive_string_replace(value, old, new)
    elif isinstance(obj, tuple):
        new_tuple = []
        for index, value in enumerate(obj):
            new_tuple.append(recursive_string_replace(value, old, new))
        obj = tuple(new_tuple)
    elif isinstance(obj, str):
        if old in obj:
            obj = obj.replace(old, new)
    return obj


In [121]:
for obj in env.objects:
    print_structure(obj.type.name)
    if obj.type.name == 'AssetBundle':
        tree = obj.read_typetree()
        recursive_string_replace(tree, 'ui_jacket_001686', 'ui_jacket_002000')
        obj.save_typetree(tree)
        
with open('./test.ab', 'wb') as f:
    f.write(env.file.save())

test_env = UnityPy.load(f'./test.ab')
for obj in test_env.objects:
    if obj.type.name == 'AssetBundle':
        tree = obj.read_typetree()
        print_structure(tree)

root: Sprite
root: AssetBundle
root: Texture2D
root.m_Name: jacket/ui_jacket_002000.ab
root.m_PreloadTable[0].m_FileID: 0
root.m_PreloadTable[0].m_PathID: -1066836184272249663
root.m_PreloadTable[1].m_FileID: 0
root.m_PreloadTable[1].m_PathID: 739899962197238906
root.m_Container[0]: ('assets/assetbundle/jacket/ui_jacket_002000.png', {'preloadIndex': 0, 'preloadSize': 2, 'asset': {'m_FileID': 0, 'm_PathID': 739899962197238906}})
root.m_Container[1]: ('assets/assetbundle/jacket/ui_jacket_002000.png', {'preloadIndex': 0, 'preloadSize': 2, 'asset': {'m_FileID': 0, 'm_PathID': -1066836184272249663}})
root.m_MainAsset.preloadIndex: 0
root.m_MainAsset.preloadSize: 0
root.m_MainAsset.asset.m_FileID: 0
root.m_MainAsset.asset.m_PathID: 0
root.m_RuntimeCompatibility: 1
root.m_AssetBundleName: jacket/ui_jacket_002000.ab
root.m_IsStreamedSceneAssetBundle: False
root.m_ExplicitDataLayout: 0
root.m_PathFlags: 7


In [122]:
def find_image_path(directory):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.svg']
    path = Path(directory)
    # 첫 번째 이미지 파일을 찾기
    for file in path.rglob('*'):
        if file.suffix.lower() in image_extensions:
            return file
    
    # 이미지 파일이 없을 경우 None 반환
    return None

In [123]:
# iterate over internal objects
for obj in env.objects:
    # process specific object types
    if obj.type.name == 'Texture2D':
        # parse the object data
        data = obj.read()
        data.image = Image.open(find_image_path('../example')).resize(data.image.size).convert('RGB').convert('RGBA')
        data.save()

with open('./test.ab', 'wb') as f:
    f.write(env.file.save())
        

# 정리 및 테스트

In [164]:
def create_new_jacket_image(in_path: str, out_path: str, music_id: str):
    for jacket_sub_name in ['', '_s']:
        env = UnityPy.load(f'../data/ui_jacket_001686{jacket_sub_name}.ab')
        for obj in env.objects:
            if obj.type.name == 'Texture2D':
                data = obj.read()
                data.image = Image.open(find_image_path(in_path)).resize(data.image.size).convert('RGB').convert('RGBA')
                data.save()
            
            tree = obj.read_typetree()
            recursive_string_replace(tree, f'_001686', f'_{music_id}')
            obj.save_typetree(tree)
        
        with open(f'{out_path}/ui_jacket_{music_id}{jacket_sub_name}.ab', 'wb') as f:
            f.write(env.file.save())


In [165]:
shutil.rmtree('./output', ignore_errors=True)

In [166]:
os.makedirs('./output' , exist_ok=True)

create_new_jacket_image('../example', './output', '002000')

In [167]:
env = UnityPy.load(f'../data/ui_jacket_001686.ab')
for obj in env.objects:
    print('-----------------------')
    tree = obj.read_typetree()
    print_structure(tree)

-----------------------
root.m_Name: UI_Jacket_001686
root.m_Rect.x: 0.0
root.m_Rect.y: 0.0
root.m_Rect.width: 400.0
root.m_Rect.height: 400.0
root.m_Offset.x: 0.0
root.m_Offset.y: 0.0
root.m_Border.x: 0.0
root.m_Border.y: 0.0
root.m_Border.z: 0.0
root.m_Border.w: 0.0
root.m_PixelsToUnits: 100.0
root.m_Pivot.x: 0.5
root.m_Pivot.y: 0.5
root.m_Extrude: 1
root.m_IsPolygon: False
root.m_RenderDataKey.first.data[0]: 1649677243
root.m_RenderDataKey.first.data[1]: 1261876480
root.m_RenderDataKey.first.data[2]: 3169460618
root.m_RenderDataKey.first.data[3]: 30573560
root.m_RenderDataKey.second: 21300000
root.m_SpriteAtlas.m_FileID: 0
root.m_SpriteAtlas.m_PathID: 0
root.m_RD.texture.m_FileID: 0
root.m_RD.texture.m_PathID: 739899962197238906
root.m_RD.alphaTexture.m_FileID: 0
root.m_RD.alphaTexture.m_PathID: 0
root.m_RD.m_SubMeshes[0].firstByte: 0
root.m_RD.m_SubMeshes[0].indexCount: 6
root.m_RD.m_SubMeshes[0].topology: 0
root.m_RD.m_SubMeshes[0].baseVertex: 0
root.m_RD.m_SubMeshes[0].firstVerte

In [168]:
r_env = UnityPy.load(f'./output/ui_jacket_002000.ab')
# r_env = UnityPy.load(f'./output/ui_jacket_002000_s.ab')

for obj in r_env.objects:
    print('-----------------------' + obj.type.name)
    tree = obj.read_typetree()
    print_structure(tree)


-----------------------Sprite
root.m_Name: UI_Jacket_002000
root.m_Rect.x: 0.0
root.m_Rect.y: 0.0
root.m_Rect.width: 400.0
root.m_Rect.height: 400.0
root.m_Offset.x: 0.0
root.m_Offset.y: 0.0
root.m_Border.x: 0.0
root.m_Border.y: 0.0
root.m_Border.z: 0.0
root.m_Border.w: 0.0
root.m_PixelsToUnits: 100.0
root.m_Pivot.x: 0.5
root.m_Pivot.y: 0.5
root.m_Extrude: 1
root.m_IsPolygon: False
root.m_RenderDataKey.first.data[0]: 1649677243
root.m_RenderDataKey.first.data[1]: 1261876480
root.m_RenderDataKey.first.data[2]: 3169460618
root.m_RenderDataKey.first.data[3]: 30573560
root.m_RenderDataKey.second: 21300000
root.m_SpriteAtlas.m_FileID: 0
root.m_SpriteAtlas.m_PathID: 0
root.m_RD.texture.m_FileID: 0
root.m_RD.texture.m_PathID: 739899962197238906
root.m_RD.alphaTexture.m_FileID: 0
root.m_RD.alphaTexture.m_PathID: 0
root.m_RD.m_SubMeshes[0].firstByte: 0
root.m_RD.m_SubMeshes[0].indexCount: 6
root.m_RD.m_SubMeshes[0].topology: 0
root.m_RD.m_SubMeshes[0].baseVertex: 0
root.m_RD.m_SubMeshes[0].firs